## Matrix Mul comparsion

### Pycuda

In [2]:
import numpy as np

import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule

In [3]:

kernel = SourceModule("""
__global__
void MatrixMul(float* A, float* B, float* C, int mid_size, int width) {
    /* 
        Matrix multiplication A * B = C
    */
    //extern __shared__ float shmem_A[BLOCK_SIZE * BLOCK_SIZE];
    //extern __shared__ float shmem_B[BLOCK_SIZE * BLOCK_SIZE];
    extern __shared__ float shmem_A[16 * 16];
    extern __shared__ float shmem_B[16 * 16];

    int col = blockIdx.x * blockDim.x + threadIdx.x;  // column num
    int row = blockIdx.y * blockDim.y + threadIdx.y;  // line num

    float res = .0f;

    for (int k = 0; k < mid_size; k += blockDim.x) {
        shmem_A[threadIdx.y * blockDim.x + threadIdx.x] = A[row * mid_size + k + threadIdx.x];
        shmem_B[threadIdx.y * blockDim.x + threadIdx.x] = B[k * width + col + threadIdx.y * width];
        __syncthreads();

        for (int j = 0; j < blockDim.x; j++) {
            res += shmem_A[threadIdx.y * blockDim.x + j] * shmem_B[j * blockDim.x + threadIdx.x];
        }
        __syncthreads();
    }

    C[row * width + col] = res;
}
""")

func = kernel.get_function("MatrixMul")


In [4]:
# A_height = 1280
# A_width = 3840

# B_height = 3840
# B_width = 2560

# C_height = 1280
# C_width = 2560

# mid_size = 3840

##################

A_height = 12800
A_width = 12800

B_height = 12800
B_width = 12800

C_height = 12800
C_width = 12800

mid_size = 12800

In [5]:
# A = np.ones(A_height * A_width, dtype=np.float32)
# B = np.ones(B_height * B_width, dtype=np.float32)

# C = np.zeros(C_height * C_width, dtype=np.float32)

In [6]:
A = np.random.randint(low=1, high=10, size=A_height * A_width).astype(np.float32)
B = np.random.randint(low=1, high=10, size=B_height * B_width).astype(np.float32)

C = np.zeros(C_height * C_width, dtype=np.float32)

In [7]:
A_matrix = A.reshape((A_height, A_width))
B_matrix = B.reshape((B_height, B_width))

In [8]:
blockSize_x = 16
blockSize_y = 16

numBlocks_x = (C_width + blockSize_x - 1) // blockSize_x
numBlocks_y = (C_height + blockSize_y - 1) // blockSize_y

numBlocks_x, numBlocks_y

(800, 800)

In [9]:
%%time

func(
    drv.In(A),
    drv.In(B),
    drv.Out(C),
    np.int32(A_width),
    np.int32(C_width),
    block=(blockSize_x, blockSize_y, 1),
    grid=(numBlocks_x, numBlocks_y),
    shared=2 * blockSize_x * blockSize_x
)

CPU times: user 8.18 s, sys: 380 ms, total: 8.56 s
Wall time: 8.58 s


In [10]:
C

array([318487., 318030., 319204., ..., 319392., 321063., 319981.],
      dtype=float32)

### Numpy

In [11]:
A_matrix = A.reshape((A_height, A_width))
B_matrix = B.reshape((B_height, B_width))

In [12]:
%%time
C_numpy = A_matrix @ B_matrix

CPU times: user 1min 38s, sys: 1.54 s, total: 1min 39s
Wall time: 58.8 s


In [13]:
C_numpy

array([[318487., 318030., 319204., ..., 319767., 320942., 320079.],
       [314165., 312124., 312666., ..., 314434., 313645., 315405.],
       [320241., 316501., 319138., ..., 321096., 322209., 321786.],
       ...,
       [318994., 315352., 317554., ..., 317301., 319059., 319545.],
       [317697., 314528., 317115., ..., 318879., 317077., 317762.],
       [318332., 317309., 318236., ..., 319392., 321063., 319981.]],
      dtype=float32)

In [14]:
print("Results form pycuda and numpy are equal:", np.all(np.ravel(C_numpy) == C))

Results form pycuda and numpy are equal: True


### Reikna

In [15]:
from pycuda.gpuarray import dot, to_gpu

from reikna.cluda import dtypes, any_api
from reikna.linalg import MatrixMul

In [16]:
api = any_api()
thr = api.Thread.create()

In [17]:
%%time

mul = MatrixMul(to_gpu(A_matrix), to_gpu(B_matrix))
dotc = mul.compile(thr)
res_dev = thr.empty_like(A_matrix)

dotc(res_dev, to_gpu(A_matrix), to_gpu(B_matrix))
output = res_dev.get()

CPU times: user 6.8 s, sys: 164 ms, total: 6.96 s
Wall time: 7.28 s


In [18]:
output

array([[318487., 318030., 319204., ..., 319767., 320942., 320079.],
       [314165., 312124., 312666., ..., 314434., 313645., 315405.],
       [320241., 316501., 319138., ..., 321096., 322209., 321786.],
       ...,
       [318994., 315352., 317554., ..., 317301., 319059., 319545.],
       [317697., 314528., 317115., ..., 318879., 317077., 317762.],
       [318332., 317309., 318236., ..., 319392., 321063., 319981.]],
      dtype=float32)

In [19]:
print("Results form reikna and numpy are equal:", np.all(C_numpy == output))

Results form reikna and numpy are equal: True


### Pytorch

In [20]:
import torch

In [21]:
a = torch.from_numpy(A_matrix)
b = torch.from_numpy(B_matrix)

In [22]:
%%time

with torch.no_grad():
    a_gpu = a.to('cuda:0')
    b_gpu = b.to('cuda:0')
    
    output = torch.matmul(a_gpu, b_gpu).cpu()

CPU times: user 2.76 s, sys: 1.13 s, total: 3.89 s
Wall time: 3.89 s


In [23]:
output = output.numpy()
output

array([[318487., 318030., 319204., ..., 319767., 320942., 320079.],
       [314165., 312124., 312666., ..., 314434., 313645., 315405.],
       [320241., 316501., 319138., ..., 321096., 322209., 321786.],
       ...,
       [318994., 315352., 317554., ..., 317301., 319059., 319545.],
       [317697., 314528., 317115., ..., 318879., 317077., 317762.],
       [318332., 317309., 318236., ..., 319392., 321063., 319981.]],
      dtype=float32)

In [24]:
print("Results form pytorch and numpy are equal:", np.all(C_numpy == output))

Results form pytorch and numpy are equal: True
